## Streaming Real Time Options Data with ThetaData

In [ ]:
import datetime as dt
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType,
    StreamResponseType,
)

Define the option contract parameters for the Iron Condor

In [ ]:
ticker = "SPY"
expiration_date = dt.date(2024, 3, 28)
long_put_strike = 460
short_put_strike = 465
short_call_strike = 480
long_call_strike = 485

Initialize global variables for option quotes and price

In [ ]:
long_put = Quote()
short_put = Quote()
short_call = Quote()
long_call = Quote()
price = 0

In [ ]:
def callback_iron_condor(msg):
    # Handle received messages
    if msg.type != StreamMsgType.QUOTE:
        return
    
    global price
    
    # Update the relevant quote based on the message
    if not msg.contract.isCall and msg.contract.strike == long_put_strike:
        long_put.copy_from(msg.quote)
    elif not msg.contract.isCall and msg.contract.strike == short_put_strike:
        short_put.copy_from(msg.quote)
    elif msg.contract.isCall and msg.contract.strike == short_call_strike:
        short_call.copy_from(msg.quote)
    elif msg.contract.isCall and msg.contract.strike == long_call_strike:
        long_call.copy_from(msg.quote)
    
    # Calculate Iron Condor bid, ask, and mid prices
    condor_bid = round(
        long_put.bid_price
        - short_put.bid_price
        + long_call.bid_price
        - short_call.bid_price,
        2,
    )
    condor_ask = round(
        long_put.ask_price
        - short_put.ask_price
        + long_call.ask_price
        - short_call.ask_price,
        2,
    )
    condor_mid = round((condor_ask + condor_bid) / 2, 2)

    # Get the timestamp from the message
    time_stamp = thetadata.client.ms_to_time(msg.quote.ms_of_day)
    
    # Print the Iron Condor prices if they have changed
    if price != condor_mid:
        print(
            f"time: {time_stamp} bid: {condor_bid} mid: {condor_mid} ask: {condor_ask}"
        )
        price = condor_mid

In [ ]:
def stream_iron_condor():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")
    
    # Connect to the data stream
    client.connect_stream(callback_iron_condor)
    
    # Request quote streams for all four legs of the Iron Condor
    lp_id = client.req_quote_stream_opt(
        ticker, expiration_date, long_put_strike, OptionRight.PUT
    )
    sp_id = client.req_quote_stream_opt(
        ticker, expiration_date, short_put_strike, OptionRight.PUT
    )
    sc_id = client.req_quote_stream_opt(
        ticker, expiration_date, short_call_strike, OptionRight.CALL
    )
    lc_id = client.req_quote_stream_opt(
        ticker, expiration_date, long_call_strike, OptionRight.CALL
    )
    
    # Verify the subscriptions
    if (
        client.verify(lp_id) != StreamResponseType.SUBSCRIBED
        or client.verify(sp_id) != StreamResponseType.SUBSCRIBED
        or client.verify(sc_id) != StreamResponseType.SUBSCRIBED
        or client.verify(lc_id) != StreamResponseType.SUBSCRIBED
    ):
        raise Exception("Unable to stream.")

Start streaming Iron Condor quotes

In [ ]:
stream_iron_condor()

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.